<a href="https://colab.research.google.com/github/mahesh-ml/MLBasics/blob/main/sentiment_analysis_yelp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers requests beautifulsoup4 pandas numpy

     |████████████████████████████████| 2.8 MB 11.9 MB/s 
     |████████████████████████████████| 636 kB 67.9 MB/s 
     |████████████████████████████████| 895 kB 92.1 MB/s 
     |████████████████████████████████| 3.3 MB 79.7 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [6]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

Downloading:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [18]:
tokens = tokenizer.encode('Its ok', return_tensors='pt')

In [19]:
result = model(tokens)

In [20]:
result

SequenceClassifierOutput([('logits',
                           tensor([[-1.7011,  0.1097,  1.9966,  0.7932, -1.1048]],
                                  grad_fn=<AddmmBackward>))])

In [21]:
int(torch.argmax(result.logits)+1)


3

In [27]:
#collect reviews
import re
r = requests.get('https://www.yelp.com/biz/mejico-sydney-2')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p',{'class':regex})
reviews = [result.text for result in results]

In [29]:
reviews

['The food is fresh and tasty. \xa0The scallop ceviche started the lunch. The scallops were tender with a great acidity and use of mango and peppers. The steak was tender and I got the hint of tequila in the sauce. I enjoyed a watermelon salad that complimented the the steak. The portions are good, but a stretch if you are sharing. My only down point is the service. They really only showed up to present my next plate and never checked to see if I wanted another drink (which I did).Enjoyed the food.',
 "Don't come here expecting legit Mexican food but a modern twist on some staples. Loud party area, fun drinks and friendly staff make this a hip meeting area for large groups. Drinks were better than the food. They stuff the families toward the back but lack any amenities (no changing table) except a high chair. Service started off friendly but it took a while to get someone to take our order and then they forgot our dish which came out cold when we asked for it. Then we had to flag someo

In [31]:
import pandas as pd
import numpy as np
df= pd.DataFrame(np.array(reviews), columns=['review'])

In [33]:
df.head()
df.tail()

,review
5,Ordered feed me for $59 along with that.. Food...
6,We came here on a Thursday night @ 5pm and by ...
7,"The best tasting cauliflower taco, mushroom ce..."
8,Have been here twice and have absolutely loved...
9,Why is it that all Mexican food outside of Mex...


In [34]:
def sentiment_score(review):
   tokens = tokenizer.encode(review, return_tensors='pt')
   result = model(tokens)
   return int(torch.argmax(result.logits))+1

In [35]:
sentiment_score(df['review'].iloc[1])

3

In [38]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [39]:
df

,review,sentiment
0,The food is fresh and tasty. The scallop cevi...,4
1,Don't come here expecting legit Mexican food b...,3
2,Out of all the restaurants that I tried in Syd...,5
3,I was pleasantly surprised at what a great job...,5
4,Really nice (upmarket) Mexican restaurant. Goo...,4
5,Ordered feed me for $59 along with that.. Food...,2
6,We came here on a Thursday night @ 5pm and by ...,4
7,"The best tasting cauliflower taco, mushroom ce...",5
8,Have been here twice and have absolutely loved...,5
9,Why is it that all Mexican food outside of Mex...,3


In [40]:
df['review'].iloc[3]

"I was pleasantly surprised at what a great job they did with Mexican food at this downtown Sydney eatery. Very friendly staff, a substantial bar area, and yummy cocktails round out this dining experience in a superb way. Try the watermelon salad, and ANY of the tacos. You won't be disappointed!"